In [331]:
import pandas as pd


import bs4 as bs
import urllib.request
import yfinance as yf



In [366]:
#note that this only shows the top 25 holdings - need to find better database 
def get_etf_holdings(shares, etf_symbol):
    
    #read the top 25 holdings in data frame
    dfs = pd.read_html("https://ycharts.com/companies/{}/holdings".format(etf_symbol),header=0)
    holdings = dfs[0]
    
    holdings.drop(["% Chg", "Price"], inplace=True, axis=1)

    #get etf price from most recent close
    etf_info = yf.Ticker(etf_symbol)
    etf_price = etf_info.info["previousClose"]
  
    #convert weight to float
    holdings["% Weight"] = holdings["% Weight"].str.rstrip("%")
    holdings["% Weight"] = holdings["% Weight"].astype('float') 
    
   
    #calculate $ owned 
    holdings["Amount Owned"] =  shares * etf_price * (holdings["% Weight"] / 100)
    
    holdings.drop(["% Weight"], inplace=True, axis=1)

    return(holdings)



In [367]:
get_etf_holdings(2, "VWO")

,Symbol,Name,Amount Owned
0,BABA,Alibaba Group Holding Ltd ADR,5.2268
1,00700,Tencent Holdings Ltd,4.2997
2,2330,Taiwan Semiconductor Manufacturing Co Ltd,2.1024
3,TSM,Taiwan Semiconductor Manufacturing Co Ltd ADR,1.2556
4,00939,China Construction Bank Corp Class H,1.1388
5,NPN,Naspers Ltd Class N,0.9344
6,01398,Industrial And Commercial Bank Of China Ltd Cl...,0.8249
7,02318,Ping An Insurance (Group) Co. of China Ltd Cla...,0.8030
8,RELIANCE,Reliance Industries Ltd,0.7008
9,03690,Meituan Dianping,0.6278


In [371]:
#input data frame with etf tickers and number of shares
#TODO add fuctionality to detect if etf or stock so we can also add single stocks to portfolio as well 


def get_portfolio_holdings(holdings):
    for i, row in holdings.iterrows():
        etf_holdings = get_etf_holdings(row["Shares"], row["Symbol"])
        
        if (i==0):
            holdings_temp = pd.DataFrame(columns=["Symbol", "Name", "Amount Owned"])
            total_invested = 0
            
        etf_info = yf.Ticker(row["Symbol"])
        etf_price = etf_info.info["previousClose"]
        total_invested = etf_price * row["Shares"] + total_invested
        
    
        holdings_temp = pd.merge(left=etf_holdings, right= holdings_temp, left_on=["Symbol", "Name"], right_on=["Symbol", "Name"], how='outer', indicator=False)

        holdings_temp["Amount Owned_x"].fillna(0, inplace=True)
        holdings_temp["Amount Owned_y"].fillna(0, inplace=True)
       
        holdings_temp["Amount Owned"] = holdings_temp["Amount Owned_x"] + holdings_temp["Amount Owned_y"] 
        holdings_temp.drop(['Amount Owned_x',  'Amount Owned_y'],axis=1, inplace=True)

        
        
    holdings_temp = holdings_temp.sort_values(by=["Amount Owned"], ascending=False)   
    holdings_temp["% of Portfolio"] = (holdings_temp["Amount Owned"] / total_invested) * 100
    
    return holdings_temp

In [372]:
df = pd.DataFrame([{'Symbol':"FTEC", 'Shares':2}, {'Symbol':"SPY",'Shares':5}, {'Symbol':"VWO",'Shares':5}])




get_portfolio_holdings(df)

,Symbol,Name,Amount Owned,% of Portfolio
25,MSFT,Microsoft Corp,111.87348,6.238066
26,AAPL,Apple Inc,105.95034,5.907792
27,AMZN,Amazon.com Inc,59.65776,3.326517
28,FB,Facebook Inc A,30.70620,1.712178
33,V,Visa Inc Class A,25.44731,1.418942
...,...,...,...,...
20,TCS,Tata Consultancy Services Ltd,0.83950,0.046811
22,VALE3,Vale SA,0.82125,0.045793
21,500696,Hindustan Unilever Ltd,0.82125,0.045793
23,QNBK,Qatar National Bank SAQ,0.78475,0.043758
